In [1]:
#importing h2o libraries
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil
import numpy as np


In [3]:
target='count'
run_time=300
pct_memory=0.95
virtual_memory=psutil.virtual_memory()
min_mem_size=int(round(int(pct_memory*virtual_memory.available)/1073741824,0))
port_no=random.randint(5555,55555)
h2o.init(strict_version_check=False,min_mem_size_GB=min_mem_size,port=port_no)

Checking whether there is an H2O instance running at http://localhost:53767..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.201-b09, mixed mode)
  Starting server from C:\Users\manog\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\manog\AppData\Local\Temp\tmp0y0o9hdx
  JVM stdout: C:\Users\manog\AppData\Local\Temp\tmp0y0o9hdx\h2o_manog_started_from_python.out
  JVM stderr: C:\Users\manog\AppData\Local\Temp\tmp0y0o9hdx\h2o_manog_started_from_python.err
  Server is running at http://127.0.0.1:53767
Connecting to H2O server at http://127.0.0.1:53767... successful.


H2O cluster uptime:,04 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.3
H2O cluster version age:,2 months and 9 days
H2O cluster name:,H2O_from_python_manog_z5i2o2
H2O cluster total nodes:,1
H2O cluster free memory:,1.917 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [6]:
#importing train file
df = h2o.import_file(path = 'train.csv')

#importing test file
df_test = h2o.import_file(path = 'test.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [22]:
def get_independent_variables(df, targ):
 C = [name for name in df.columns if name != targ]
# determine column types
 ints, reals, enums,strings = [], [], [], []
 for key, val in df.types.items():
     if key in C:
         if val == 'enum':
             enums.append(key)
         elif val == 'int':
             ints.append(key)
         elif val == 'string':
            strings.append(key)
         else:
             reals.append(key)
 x=ints+enums+strings+reals
 return x

In [24]:
X=get_independent_variables(df, target)
print(X)
y = target

['season', 'holiday', 'workingday', 'weather', 'humidity', 'casual', 'registered', 'datetime', 'temp', 'atemp', 'windspeed']


In [25]:
aml = H2OAutoML(max_runtime_secs=run_time,exclude_algos = ['DeepLearning'])
aml = H2OAutoML(max_runtime_secs=run_time,exclude_algos = ['DeepLearning'])
aml.train(x=X,y=y,training_frame=df)
print(aml.leaderboard)

AutoML progress: |████████████████████████████████████████████████████████| 100%


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_1_AutoML_20190403_213948,14.3184,3.78396,14.3184,2.41072,0.0569215
GBM_1_AutoML_20190403_223236,17.4987,4.18314,17.4987,2.65311,0.0670842
GBM_2_AutoML_20190403_213948,18.4736,4.2981,18.4736,2.68488,0.0597701
GBM_3_AutoML_20190403_213948,20.2788,4.5032,20.2788,2.78564,0.0601604
GBM_2_AutoML_20190403_223236,20.6009,4.53882,20.6009,2.95725,0.0905501
GBM_3_AutoML_20190403_223236,22.0129,4.69179,22.0129,2.97858,0.076483
GLM_grid_1_AutoML_20190403_213948_model_1,34.3276,5.85898,34.3276,4.25956,0.248612
GLM_grid_1_AutoML_20190403_223236_model_1,34.3276,5.85898,34.3276,4.25956,0.248612
GBM_4_AutoML_20190403_213948,40.4256,6.35811,40.4256,4.16274,nan
GBM_5_AutoML_20190403_213948,55.2706,7.43442,55.2706,3.7346,nan


In [26]:
best_model = h2o.get_model(aml.leaderboard[0,'model_id'])
pred = best_model.predict(df_test)
df_res = pd.DataFrame(h2o.as_list(df_test), columns=['datetime'])
df_res['count'] = h2o.as_list(pred)
df_res.to_csv('resulth2o.csv', index=False)

gbm prediction progress: |████████████████████████████████████████████████| 100%


C:\Users\manog\Anaconda3\lib\site-packages\h2o\job.py:69: UserWarning: Test/Validation dataset is missing column 'casual': substituting in a column of NaN
  warnings.warn(w)
C:\Users\manog\Anaconda3\lib\site-packages\h2o\job.py:69: UserWarning: Test/Validation dataset is missing column 'registered': substituting in a column of NaN
  warnings.warn(w)
